---
# Data Cleaning
---
In this notebook, it is the data cleaning section of the project.

1. Each column type and it ranges of values/category will be analyzed.
2. Columns that provide no valuable information will be discarded.
3. Missing values will be found and replaced.
4. Categories that are highly correleated to another column will be replaced, inorder to no pass the same information twice to the classification algorithm.


## The columns available in the dataset and their description:

| Column name       | Description                                                                                            | Needed for further analysis   |
| ---------------- | ------------------------------------------------------------------------------------------------------- | :--------------------------: |
| CustomerID       | ID unique to each customer                                                                              | No                           |
| Count            | A value used in reporting/dashboarding to sum up the number of customers in a filtered set.             | No                           |
| Country          | Country of residence                                                                                    | No - US only - Need at least 2 value for decision tree                   |
| State            | State of residence                                                                                      | No - California only - Need at least 2 value for decision tree                             |
| City             | City of residence                                                                                       | Not sure - Dont think so - 1129 diferent values                             |
| Zip Code         | Zip Code of residence                                                                                   | Maybe - either keep Zip Code or latitude, longitude                             |
| Lat Long         | Latitude and longitude coordinates of residence                                                         | Maybe - if we intend to calculate distance from some landmark |
| Latitude         | Latitude of residence                                                                                   | Maybe - either keep Zip Code or latitude, longitude                             |
| Longitude        | Longitude of residence                                                                                  | Maybe - either keep Zip Code or latitude, longitude                              |
| Gender           | Customer's gender                                                                                       | Yes                              |
| Senior Citizen   | Is customer 65 or older?                                                                                | Yes                              |
| Partner          | Does customer have a partner?                                                                           | Yes                              |
| Dependents       | Does customer live with any dependents (child, parents, grandparents)?                                  | Yes                               |
| Tenure Months    | Total amount of months that the customer has been with the company by the end of the quarter            | Yes                          |
| Phone Service    | Does customer have a home phone service with the company?                                               | Yes                              |
| Multiple Lines   | Does the customer have multiple telephone lines with the company?                                       | Yes                              |
| Internet Service | Does the customer have internet service with the company?                                               | Yes                          |
| Online Security  | Has the customer subscribed to an additional online security service provided by the company?           | Yes                              |
| Online Backup    | Has the customer subscribed to an additional online backup service provided by the company?             | Yes                              |
| Device Protection | Has the customer subscribed to an additional device protection plan for their internet equipment?      | Yes                              |
| Tech Support     | Has the customer subscribed to an additional technical support plan with reduced wait times?            | Yes                               |
| Streaming TV     | Does customer use their Internet service to stream television programming from a third-party provider?  | Yes - Streaming TV and Streaming Movies are different                             |
| Streaming Movies | Does customer use their Internet service to stream movies from a third-party provider?                  | Yes - Streaming TV and Streaming Movies are different                              |
| Contract         | Current contract type: Month-to-Month, One Year, Two Year                                               | Yes                          |
| Paperless Billing | Has customer chosen paperless billing?                                                                 | Yes                              |
| Payment Method   | How the customer pays their bill: Bank Withdrawal, Credit Card, Mailed Check                            | Yes                              |
| Monthly Charges  | Customer’s current total monthly charge for all their services from the company.                        | Yes                          |
| Total Charges    | Customer’s total charges, calculated to the end of the quarter.                                         | No - Using Montlhy Charges should be sufficient                          |
| Churn Label      | Yes = the customer left the company this quarter. No = the customer remained with the company.          | No - Same as Churn Value - Keeping Churn Value only                          |
| Churn Value      | 1 = the customer left the company this quarter. 0 = the customer remained with the company.             | Yes                          |
| Churn Score      | A value from 0-100 that is calculated using the predictive tool IBM SPSS Modeler. The higher the score, the more likely the customer will churn. | Yes - For validation to compare our model performance against theirs                              |
| CLTV            | Customer Lifetime Value. A predicted CLTV is calculated using corporate formulas and existing data. The higher the value, the more valuable the customer. High value customers should be monitored for churn.                                                                                  |  Yes - For validation to compare our model performance against theirs                             |
| Churn Reason    | Customer’s specific reason for leaving the company. Directly related to Churn Category                   | Maybe - Depends on the amount of different categories                       |




---

### Importing necessary library

In [1]:
import pandas as pd
import numpy as np

### Read data file

In [2]:
churn_df = pd.read_excel('../data/churn.xlsx')
churn_df

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,1,United States,California,Landers,92285,"34.341737, -116.539416",34.341737,-116.539416,Female,...,Two year,Yes,Bank transfer (automatic),21.15,1419.4,No,0,45,5306,NaN
7039,6840-RESVB,1,United States,California,Adelanto,92301,"34.667815, -117.536183",34.667815,-117.536183,Male,...,One year,Yes,Mailed check,84.80,1990.5,No,0,59,2140,NaN
7040,2234-XADUH,1,United States,California,Amboy,92304,"34.559882, -115.637164",34.559882,-115.637164,Female,...,One year,Yes,Credit card (automatic),103.20,7362.9,No,0,71,5560,NaN
7041,4801-JZAZL,1,United States,California,Angelus Oaks,92305,"34.1678, -116.86433",34.167800,-116.864330,Female,...,Month-to-month,Yes,Electronic check,29.60,346.45,No,0,59,2793,NaN


### Looking at the data type of the dataset columns


In [3]:
churn_df.dtypes

CustomerID            object
Count                  int64
Country               object
State                 object
City                  object
Zip Code               int64
Lat Long              object
Latitude             float64
Longitude            float64
Gender                object
Senior Citizen        object
Partner               object
Dependents            object
Tenure Months          int64
Phone Service         object
Multiple Lines        object
Internet Service      object
Online Security       object
Online Backup         object
Device Protection     object
Tech Support          object
Streaming TV          object
Streaming Movies      object
Contract              object
Paperless Billing     object
Payment Method        object
Monthly Charges      float64
Total Charges         object
Churn Label           object
Churn Value            int64
Churn Score            int64
CLTV                   int64
Churn Reason          object
dtype: object

---
### Looking at the the values of the categorical columns
---

**Country column**

Conclusion : Only one value is found in this column. There is no value to a column containing a single value. Thus, this column can be discarded.

In [4]:
print(churn_df['Country'].unique())

['United States']


**State column**

Conclusion : Only one value is found in this column. There is no value to a column containing a single value. Thus, this column can be discarded.

In [5]:
print(churn_df['State'].unique())

['California']


**City column**

Conclusion : 1129 cities is found in the City Column. It will be kept in case, we want to use it along the way.

In [6]:
print(churn_df['City'].unique())

print(len(churn_df['City'].unique()))

['Los Angeles' 'Beverly Hills' 'Huntington Park' ... 'Standish' 'Tulelake'
 'Olympic Valley']
1129


**Zip Code column**

Conclusion : 1652 zip code is found in the Zip Code Column. It will be kept in case, we want to use it along the way.

In [7]:
print(churn_df['Zip Code'].unique())

print(len(churn_df['Zip Code'].unique()))

[90003 90005 90006 ... 96128 96134 96146]
1652


**Partner column**

Conclusion : Simple Yes/No column

In [8]:
print(churn_df['Partner'].unique(), len(churn_df['Partner'].unique()))

['No' 'Yes'] 2


**Churn Reason column**

Conclusion : 21 different reasons is found in the Churn Reason Column. This column will be useful to determine which features could be the best predictors to forecast clients leaving the company.
  * The 21 different reasons can be grouped into 7 categories:
    - Competitors
    - Price
    - Dissatifaction
    - Moved
    - Not Sure
    - Deceased
    - Not applicable
    

In [9]:
print(churn_df['Churn Reason'].unique())

print(len(churn_df['Churn Reason'].unique()))

['Competitor made better offer' 'Moved' 'Competitor had better devices'
 'Competitor offered higher download speeds'
 'Competitor offered more data' 'Price too high' 'Product dissatisfaction'
 'Service dissatisfaction' 'Lack of self-service on Website'
 'Network reliability' 'Limited range of services'
 'Lack of affordable download/upload speed' 'Long distance charges'
 'Extra data charges' "Don't know" 'Poor expertise of online support'
 'Poor expertise of phone support' 'Attitude of service provider'
 'Attitude of support person' 'Deceased' nan]
21


**Streaming TV**

Conclusion : Simple Yes/No/No internet column

In [10]:
churn_df['Streaming TV'].unique()

array(['No', 'Yes', 'No internet service'], dtype=object)

**Streaming Movies**

Conclusion : Simple Yes/No/No internet column

In [11]:
churn_df['Streaming Movies'].unique()

array(['No', 'Yes', 'No internet service'], dtype=object)

**Are Streaming TV and Streaming Movies columns different?**

Conclusion : Yes. Thus, we need to keep both for for the data analysis process.

In [12]:
# Looking if the columns 'Streaming TV' and Streaming Movies' are identical
churn_df[churn_df['Streaming TV'] != churn_df['Streaming Movies']].loc[:,'Streaming TV':'Streaming Movies']

,Streaming TV,Streaming Movies
6,No,Yes
14,Yes,No
19,Yes,No
25,Yes,No
31,No,Yes
...,...,...
7020,Yes,No
7021,Yes,No
7029,No,Yes
7031,Yes,No


---
### Removing unecessary columns
---

**CustomerID** : ID unique to each customer.
 - Brings no value

**Count** : A value used in reporting/dashboarding to sum up the number of customers in a filtered set.
 - Brings no value

**Country** : Contains a single value.
  - A column needs to contain at least two different values in order to be used in Regression/Decision Trees

**State** : Contains a single value.
 - A column needs to contain at least two different values in order to be used in Regression/Decision Trees

**Lat Long** : A coordinate cannot be use as an input for Regression/Decision Tree. Also, we are already keepin the column latitude and longitude.
 - Redundent

---

In [13]:
churn_df.drop(columns=['CustomerID',
                       'Count',
                       'Country',
                       'State',
                       'Lat Long',
                       'Total Charges',
                       'Churn Label'
                      ], inplace=True)

churn_df.head(5)

,City,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,...,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Churn Value,Churn Score,CLTV,Churn Reason
0,Los Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,1,86,3239,Competitor made better offer
1,Los Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,...,No,No,Month-to-month,Yes,Electronic check,70.70,1,67,2701,Moved
2,Los Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,Yes,...,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,1,86,5372,Moved
3,Los Angeles,90010,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,...,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,1,84,5003,Moved
4,Los Angeles,90015,34.039224,-118.266293,Male,No,No,Yes,49,Yes,...,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,1,89,5340,Competitor had better devices


---
### Investigating missing values
---

**Finding the missing values**

In [14]:
churn_df.isna().sum()

City                    0
Zip Code                0
Latitude                0
Longitude               0
Gender                  0
Senior Citizen          0
Partner                 0
Dependents              0
Tenure Months           0
Phone Service           0
Multiple Lines          0
Internet Service        0
Online Security         0
Online Backup           0
Device Protection       0
Tech Support            0
Streaming TV            0
Streaming Movies        0
Contract                0
Paperless Billing       0
Payment Method          0
Monthly Charges         0
Churn Value             0
Churn Score             0
CLTV                    0
Churn Reason         5174
dtype: int64

**Are the missing values of the Churn Reason columns are only associated with remaining clients?**

Conslusion: Yes 

In [15]:
idx_stayed = set(churn_df[churn_df['Churn Value'] == 0].index)

idx_na = set(churn_df['Churn Reason'].isna().index)

# Looking at whether the index of the missing values of the column 'Churn Reason'
# are the same as the index of the client that stayed
if idx_stayed.difference(idx_na) == set() :
    print('All the missing values of the Churn Reason column belongs to remaining clients.')
else:
    print('The missing values of the Churn Reason column can be found for both churn and unchurn clients.')

All the missing values of the Churn Reason column belongs to remaining clients.


### Replacing missing values

In [16]:
# Replacing missing value for remaining customer with 'Not applicable'
churn_df['Churn Reason'] = churn_df['Churn Reason'].fillna('Not applicable')


In [17]:
# Looking that there is no more missing values
churn_df.isna().sum()

City                 0
Zip Code             0
Latitude             0
Longitude            0
Gender               0
Senior Citizen       0
Partner              0
Dependents           0
Tenure Months        0
Phone Service        0
Multiple Lines       0
Internet Service     0
Online Security      0
Online Backup        0
Device Protection    0
Tech Support         0
Streaming TV         0
Streaming Movies     0
Contract             0
Paperless Billing    0
Payment Method       0
Monthly Charges      0
Churn Value          0
Churn Score          0
CLTV                 0
Churn Reason         0
dtype: int64

---
### Replacing similar value in column 'Churn Reason'

This column will be useful to determine which features could be the best predictors to forecast clients leaving the company.

There are 21 different reasons in the Churn Reason Column.

The 21 different reasons can be grouped into 7 categories:
  - Competitors
  - Price
  - Dissatifaction
  - Moved
  - Not Sure
  - Deceased
  - Not applicable

---

In [18]:
# Create dict for string replacement
repl_dict = {'Competitors' : ['Competitor made better offer',
                              'Competitor had better devices',
                              'Competitor offered higher download speeds',
                              'Competitor offered more data'
                             ],
             
             'Price' : ['Price too high',
                        'Lack of affordable download/upload speed',
                        'Long distance charges',
                        'Extra data charges' 
                        ],
             
             'Dissactisfaction' : ['Product dissatisfaction',
                                   'Service dissatisfaction',
                                   'Lack of self-service on Website',
                                   'Network reliability', 
                                   'Limited range of services',
                                   'Poor expertise of online support',
                                   'Poor expertise of phone support',
                                   'Attitude of service provider',
                                   'Attitude of support person'
                                   ]
            }

# Reverse the order of the dict to be able to use replace
repl_dict_flatten = {orig_str : repl_str for repl_str, orig_str_list in repl_dict.items() for orig_str in orig_str_list}

# Replace strings
churn_df['Churn Reason'] = churn_df['Churn Reason'].replace(repl_dict_flatten)

# # Verifying that the replacement was done correctly
churn_df['Churn Reason'].unique()

array(['Competitors', 'Moved', 'Price', 'Dissactisfaction', "Don't know",
       'Deceased', 'Not applicable'], dtype=object)

---

### Removing highly correlated data

---
At first, we kept all values of categorical data. Hovever, when the correlation heatmap was produced, we could observe some perfectly correlated columns.

All the categorical columns that contains a category 'No internet service' is in fact a perfect replica of the columns 'Internet Service' with the category value of 'No'.

Therefore, we will be removing the 'No internet service' value by replacing it with 'No'.


![Correlation Heatmap](../graph/Correlation_heatmap_all.png)

---

In [19]:
# Setting the columns in which we want to proceed with the replacement
col = ['Online Security', 'Online Backup',
       'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies']

# Replacing 'No internet Service' by 'No'
churn_df[col] = churn_df[col].replace({'No internet service':'No'})

# Verify that the replacement was done correctly
churn_df['Online Security'].unique()

array(['Yes', 'No'], dtype=object)

In [20]:
# Setting the columns in which we want to proceed with the replacement
col = ['Multiple Lines']

# Replacing 'No internet Service' by 'No'
churn_df[col] = churn_df[col].replace({'No phone service':'No'})

# Verify that the replacement was done correctly
churn_df['Online Security'].unique()

array(['Yes', 'No'], dtype=object)

---
### Write cleaned data file
---

In [21]:
churn_df.to_excel('../data/churn_cleaned.xlsx', index=False)